# Load Graphs

In [2]:
import os
import glob

path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"

def load_graphs(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'*.{}'.format("edgelist"))))      
    return path_list



## Load Graph for t=1

In [14]:
import networkx as nx

paths=load_graphs(path)
count=len(paths[0])
number=len(paths)

def create_network_t1(paths:list,count:int,number:int):
    edgelist=paths[number-1][count-1]
    G=nx.read_edgelist(edgelist)
    return G


In [15]:
G=create_network_t1(paths,count,number)
print(G)

Graph with 100 nodes and 1522 edges


## Load Graph for t > 1

In [16]:
def create_network_t(paths:list,t:int,run:int):
    G=None
    for i in range(t):
        edgelist=paths[run][i]
        g=nx.read_edgelist(edgelist)
        if G is None:
            G=g
        else:
            G=nx.compose(G,g)
    return G

In [17]:
G=create_network_t(paths,4,3)
print(G)

Graph with 100 nodes and 3133 edges


# Load model and agent data

In [20]:
def load_agent_data(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'agent_data.{}'.format("csv"))))      
    return path_list

agent_paths=load_agent_data(path)

def load_model_data(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'agent_data.{}'.format("csv"))))      
    return path_list

model_paths=load_model_data

In [32]:
import pandas as pd

def read_csv_t(paths:list,run:int):
    path="%s".join(paths[run])
    df=pd.read_csv(path)
    return df
agent_df=read_csv_t(agent_paths,4)


['/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/14/agent_data.csv']


,Step,AgentID,Info,Avg_PersonalizedTrust,GeneralizedTrust,Wealth,Suspectability,ID,SecurityLevel,Partner,Memory,Threshold
0,2,50,0.000000,-0.602916,-0.959783,-0.558221,-0.782422,50,2.519678,39,9,0.539936
1,2,44,0.000000,-0.118134,-0.143073,1.744169,0.354541,44,-4.999263,86,8,0.546276
2,2,42,0.000000,0.021926,0.018317,2.126705,0.423461,42,0.497191,41,3,-0.108775
3,2,17,0.000000,-0.365221,-0.370438,-0.930478,1.393606,17,-0.644068,22,4,0.211719
4,2,53,0.000000,-0.134148,-0.180186,-3.302550,1.491366,53,10.031518,76,3,0.463998
...,...,...,...,...,...,...,...,...,...,...,...,...
9895,100,67,0.000000,-0.110519,-0.223556,-166.439622,0.593146,67,3.805931,10,6,-0.352132
9896,100,72,0.330465,-0.135897,-0.953190,-179.263123,0.670971,72,2.432102,19,7,0.743006
9897,100,8,0.348150,-0.338595,-0.916403,-178.992577,0.943512,8,-1.695737,41,9,0.317800
9898,100,39,0.000000,0.127007,0.546754,-169.074122,0.999815,39,-2.973734,51,8,0.408411


# Sparsity/Clustering

In [5]:

nx.average_clustering(G)

0.2972527536619025